In [1]:
import os
import pandas as pd
import xgboost
import utils

In [2]:
SIMPLE_FEATURE_COLUMNS = ['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]',
       'avg_cs[1]', 'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_Y[1]',
       'Lextra_Y[2]', 'Lextra_Y[3]', 'NShared', 'Mextra_DX2[0]',
       'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]', 'Mextra_DY2[0]',
       'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]', 'FOI_hits_N', 'PT', 'P']

In [3]:
DATA_PATH = "../MuID_Run_III/IDAO"

In [4]:
train = pd.concat([
    pd.read_csv(os.path.join(DATA_PATH, "train_part_%i.csv.gz" % i), usecols=SIMPLE_FEATURE_COLUMNS+["label", "weight"])
    for i in (1, 2)], axis=0, ignore_index=True)

In [5]:
test = pd.read_csv(os.path.join(DATA_PATH, "test_public.csv.gz"))

In [6]:
train.head()

,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,MatchedHit_TYPE[0],...,Mextra_DX2[3],Mextra_DY2[0],Mextra_DY2[1],Mextra_DY2[2],Mextra_DY2[3],FOI_hits_N,PT,P,label,weight
0,47,31,13,15,2.000000,1.580645,1.153846,1.133333,8,2,...,4295.08540,333.436300,951.68540,2125.09990,4145.18160,6,1764.925899,12646.817591,1,2.226325
1,92,19,11,26,2.750000,2.789474,1.363636,1.230769,8,2,...,777.42914,64.267190,185.53355,413.95690,793.56604,4,4511.281266,28556.630027,1,2.127109
2,100,21,11,12,2.930000,2.428571,1.181818,1.083333,8,2,...,1931.48400,159.182330,440.46994,979.69520,1899.83920,9,1062.550086,17491.702634,1,4.305235
3,74,27,13,20,3.067568,2.518518,1.923077,1.300000,8,2,...,443.21832,36.024845,103.41968,229.75415,438.86194,5,4991.074825,37192.868559,1,1.605583
4,18,22,8,8,1.833333,1.954546,1.125000,1.750000,8,2,...,2430.53470,190.557480,551.34094,1240.64840,2402.91920,6,1347.704888,16688.746174,1,2.055900


In [7]:
test.head()

,id,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,...,FOI_hits_Y,FOI_hits_Z,FOI_hits_DX,FOI_hits_DY,FOI_hits_DZ,FOI_hits_T,FOI_hits_DT,FOI_hits_S,PT,P
0,0,55,12,10,15,2.127273,1.166667,1.500000,1.600000,8,...,[-1849.3373 -1826.5162 -1833.7968 -1982.7971 -...,[15396.441 16315.722 16404.695 16521.16 17597...,[12.75 13.75 13.75 13.75 59. 63. ],[63.078957 68.078926 68.078926 68.078926 73.07...,[33.727364 33.74537 33.74537 33.74537 33.76...,[ 6 13 2 2 7 3],[-1 0 -2 -1 0 0],[0 1 1 1 2 3],1174.484530,6884.502272
1,1,43,18,12,15,2.697674,1.111111,1.333333,1.600000,8,...,[-383.5468 -222.24156 -240.82657 -240.82657 -...,[15401.72 15117.299 16321.432 16321.432 17514...,[ 6.375 6.375 6.875 82.5 29.5 29.5 31....,[126.27855 31.479162 33.979145 33.979145 ...,[33.95495 33.613575 33.622578 33.622578 33.63...,[ 6 4 5 9 12 6 3],[ 6 0 -2 9 -1 -1 0],[0 0 1 1 2 2 3],4344.639775,48769.526891
2,2,90,34,19,29,1.800000,2.058824,1.210526,1.344828,8,...,[-1628.1324 -1758.3168 -1887.1799 -2014.6083],[15112.236 16315.968 17508.303 18715.945],[ 25.5 27.5 118. 126. ],[126.27855 136.27849 146.27841 156.27835],[33.95495 33.99096 34.026966 34.062977],[ 3 10 7 7],[-1 0 0 0],[0 1 2 3],4109.805750,20296.048412
3,3,141,43,15,16,4.609929,2.744186,1.466667,1.125000,8,...,[372.85458 537.1497 483.45386 483.45386 520.7...,[15119.441 15321.034 15119.84 15119.84 16324...,[ 3.2083333 76.5 3.2083333 3.2083333 ...,[126.27855 31.479162 15.679264 15.679264 ...,[33.95495 33.613575 33.55668 33.55668 33.56...,[14 13 3 4 5 6 8 7 3],[14 13 1 2 1 0 2 7 0],[0 0 0 0 1 2 2 2 3],940.242439,26062.046122
4,4,22,18,8,9,1.636364,2.444444,1.000000,1.555556,8,...,[264.6528 367.2695 284.41754 305.50333 328.0...,[15203.053 16323.622 16407.324 17600.2 18808...,[ 3.2083333 82.5 3.4583333 14.833333 1...,[15.679264 33.979145 16.929256 18.179249 19.42...,[33.55668 33.622578 33.56118 33.56568 33.57...,[ 8 5 7 5 2 13 5 6],[ 1 5 0 5 2 13 5 6],[0 1 1 2 3 3 3 3],1386.377851,59872.343332


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_part, validation = train_test_split(train, test_size=0.25, shuffle=True)

In [10]:
model = xgboost.XGBClassifier(n_jobs=-1)

In [11]:
model.fit(train_part.loc[:, SIMPLE_FEATURE_COLUMNS].values, train_part.label.values,
          sample_weight=train_part.weight.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [12]:
validation_predictions = model.predict_proba(validation.loc[:, SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [13]:
utils.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

0.5579729270554931

In [14]:
model.fit(train.loc[:, SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.weight)

[22:11:59] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [15]:
predictions = model.predict_proba(test.loc[:, SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [16]:
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "sample_submission.csv", index_label="id")